<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4e2b05c9d9868d209b4ea9bdef7b88cf3605f4269b7f1e4f4c206834e0ee9780
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-02 09:44:01
-------------------
qualified stocks: 92
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.17 L
Current:  1.39 C
-------------------
Today PnL: 67.46 K (0.49%)
Current PnL: -18.21 L (-12.25%)
CY Booked + Current PnL: -9.17 L (-6.17%)
-------------------
Total profit:  3.35 L
Total loss:  -21.56 L
-------------------
Total Booked + Current PnL: 17.81 L (14.64%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.5%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.88 L (57.46%)
Deployed:  1.22 C
Current:  1.39 C
CAGR/XIRR %: 8.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-16.60,61.0,H-LC,8.17,114760.0,10448.0,6197.0,-0.33,10.02,5.40,15.96,9.0,1.69,0.84,28.61,X5K,ATH,METALS
53,MEDANTA,1486.00,14.09,60.0,H-MC,15.45,162783.0,36583.0,9588.0,0.52,28.99,5.89,36.59,108.0,3.82,1.19,45.19,XY24,NTT,HEALTHCARE
26,DIXON,18940.15,-29.60,68.0,H-MC,6.66,140936.0,26000.0,10584.0,0.20,22.62,7.51,31.83,81.0,2.46,1.03,46.08,X40N,ATH,IT
82,TTKPRESTIG,770.00,102.77,63.0,M-SC,7.35,87364.0,-13413.0,13506.0,-1.09,-13.31,15.46,0.09,245.0,-0.99,0.64,14.78,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-15.63,55.0,H-LC,8.66,168663.0,12486.0,13611.0,-0.07,7.99,8.07,16.71,22.0,0.92,1.23,26.83,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,731.35,4.41,43.0,H-SC,11.08,124614.0,-683.0,59690.0,0.52,-0.55,47.90,47.09,141.0,-0.01,0.91,55.96,MH,ATH,POWER
59,RAJOOENG,143.33,-32.41,40.0,H-SC,10.82,102290.0,-210.0,41039.0,1.38,-0.20,40.12,39.83,114.0,-0.01,0.75,1.38,AR,ATH,MISC
32,HAVELLS,2069.16,0.20,63.0,H-MC,2.62,248383.0,652.0,74416.0,1.14,0.26,29.96,30.30,92.0,0.01,1.81,13.59,X40,ATH,ELECTRICAL
38,ICICIGI,2260.25,-21.27,34.0,H-MC,4.66,135339.0,-794.0,31926.0,-0.62,-0.58,23.59,22.86,91.0,-0.02,0.99,11.96,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,0.65,41.0,H-LC,9.97,229770.0,-12154.0,75433.0,-0.33,-5.02,32.83,26.15,8.0,-0.16,1.68,8.58,X40,ATH,IT
1,ABB,7934.00,-41.13,44.0,H-LC,13.64,246096.0,-15523.0,134738.0,-0.01,-5.93,54.75,45.57,7.0,-0.12,1.80,4.72,AR,NTT,ELECTRICAL
47,JIOFIN,387.00,-0.42,44.0,H-LC,14.71,220325.0,2919.0,50565.0,0.22,1.34,22.95,24.61,48.0,0.06,1.61,56.90,XY24,BTT,FINANCE
61,RELIANCE,1533.00,-14.04,45.0,H-LC,7.86,214968.0,4602.0,24184.0,1.78,2.19,11.25,13.68,37.0,0.19,1.57,19.07,XY25,NTT,REFINERIES
45,ITC,452.00,-38.34,46.0,H-LC,3.24,198558.0,-1580.0,22477.0,0.05,-0.79,11.32,10.44,4.0,-0.07,1.45,4.73,X40,NTT,FMCG


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.20,-5.78,53.0,H-LC,2.52,233842.0,-12309.0,91526.0,-0.01,-5.00,39.14,32.18,16.0,-0.13,1.71,29.24,X200,ATH,IT
55,NESTLEIND,1377.00,-9.62,60.0,H-LC,2.85,278156.0,12730.0,44060.0,1.22,4.80,15.84,21.40,11.0,0.29,2.03,12.23,XY25,NTT,FMCG
45,ITC,452.00,-38.34,46.0,H-LC,3.24,198558.0,-1580.0,22477.0,0.05,-0.79,11.32,10.44,4.0,-0.07,1.45,4.73,X40,NTT,FMCG
20,CIPLA,1795.00,-18.75,60.0,H-LC,4.94,216816.0,12316.0,29097.0,-0.29,6.02,13.42,20.25,10.0,0.42,1.58,15.28,X40N,BTT,PHARMA
88,VBL,671.64,-11.61,49.0,H-LC,7.62,318330.0,2488.0,110174.0,0.73,0.79,34.61,35.67,5.0,0.02,2.33,14.75,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAMMAANCAP,326.0,-168.18,62.0,M-SC,8.11,78432.0,-23778.0,117170.0,4.22,-23.26,149.39,91.37,208.0,-0.20,0.57,27.32,XY25,NTT,FINANCE
58,RAJESHEXPO,518.0,1653.79,46.0,L-SC,8.78,47864.0,-89313.0,89405.0,3.74,-65.11,186.79,0.07,267.0,-1.00,0.35,19.04,OX40N,NTT,JEWELLERY
29,GREENPANEL,537.0,208.14,55.0,M-SC,7.50,137518.0,-45560.0,124536.0,3.30,-24.89,90.56,43.14,230.0,-0.37,1.00,29.36,XY24,NTT,MISC
22,COFFEEDAY,80.0,-37.63,69.0,L-SC,31.42,93562.0,-19987.0,60039.0,3.20,-17.60,64.17,35.27,268.0,-0.33,0.68,128.99,XR,NTT,HOTELS
24,DABUR,735.0,-3.84,62.0,H-MC,3.68,209372.0,12344.0,77279.0,2.52,6.27,36.91,45.49,102.0,0.16,1.53,20.12,XY24,BTT,FMCG


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDIAMART,4850.92,-51.28,47.0,H-SC,12.54,136024.0,12688.0,121075.0,-1.30,10.29,89.01,108.45,119.0,0.10,0.99,34.94,AR,ATH,MISC
82,TTKPRESTIG,770.00,102.77,63.0,M-SC,7.35,87364.0,-13413.0,13506.0,-1.09,-13.31,15.46,0.09,245.0,-0.99,0.64,14.78,OX40N,NTT,DURABLES
86,VAIBHAVGBL,521.00,40.50,47.0,H-SC,12.14,93744.0,-58247.0,132367.0,-0.96,-38.32,141.20,48.77,125.0,-0.44,0.68,9.11,XR,NTT,JEWELLERY
72,SONACOMS,1006.00,-37.22,46.0,M-SC,13.56,78199.0,-22960.0,97850.0,-0.88,-22.70,125.13,74.03,202.0,-0.23,0.57,10.27,AR,BTT,AUTO
43,INFY,2275.00,-18.52,48.0,H-LC,11.48,317285.0,4397.0,167304.0,-0.65,1.41,52.73,54.87,3.0,0.03,2.32,8.15,X40,BTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TTKPRESTIG,770.0,102.77,63.0,M-SC,7.35,87364.0,-13413.0,13506.0,-1.09,-13.31,15.46,0.09,245.0,-0.99,0.64,14.78,OX40N,NTT,DURABLES
83,UJJIVANSFB,60.0,84.36,45.0,H-SC,14.10,116478.0,-26001.0,45520.0,-0.39,-18.25,39.08,13.70,163.0,-0.57,0.85,37.48,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.16,55.0,H-SC,3.65,219501.0,-50166.0,86505.0,0.94,-18.60,39.41,13.47,138.0,-0.58,1.60,11.02,XY24,NTT,PAINTS
19,CERA,9475.0,-20.76,40.0,H-SC,3.17,112941.0,-30862.0,57611.0,-0.20,-21.46,51.01,18.60,149.0,-0.54,0.83,24.63,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,108.15,40.0,M-SC,2.91,136400.0,-38159.0,38192.0,0.18,-21.86,28.00,0.02,221.0,-1.00,1.00,18.64,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,4998.00,-26.44,50.0,H-LC,14.88,292772.0,-53184.0,177039.0,0.06,-15.37,60.47,35.80,1.0,-0.30,2.14,3.72,X40,BTT,IT
43,INFY,2275.00,-18.52,48.0,H-LC,11.48,317285.0,4397.0,167304.0,-0.65,1.41,52.73,54.87,3.0,0.03,2.32,8.15,X40,BTT,IT
45,ITC,452.00,-38.34,46.0,H-LC,3.24,198558.0,-1580.0,22477.0,0.05,-0.79,11.32,10.44,4.0,-0.07,1.45,4.73,X40,NTT,FMCG
88,VBL,671.64,-11.61,49.0,H-LC,7.62,318330.0,2488.0,110174.0,0.73,0.79,34.61,35.67,5.0,0.02,2.33,14.75,X40N,ATH,FMCG
1,ABB,7934.00,-41.13,44.0,H-LC,13.64,246096.0,-15523.0,134738.0,-0.01,-5.93,54.75,45.57,7.0,-0.12,1.80,4.72,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.09,60.0,H-MC,15.45,162783.0,36583.0,9588.0,0.52,28.99,5.89,36.59,108.0,3.82,1.19,45.19,XY24,NTT,HEALTHCARE
34,HEROMOTOCO,5949.07,-1.09,74.0,H-MC,18.53,183172.0,32259.0,25040.0,-0.20,21.38,13.67,37.97,93.0,1.29,1.34,52.09,AR,ATH,AUTO
26,DIXON,18940.15,-29.60,68.0,H-MC,6.66,140936.0,26000.0,10584.0,0.20,22.62,7.51,31.83,81.0,2.46,1.03,46.08,X40N,ATH,IT
27,DMART,5201.00,-7.44,69.0,H-LC,13.94,231064.0,46583.0,23776.0,-0.41,25.25,10.29,38.14,38.0,1.96,1.69,38.53,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.09,60.0,H-MC,15.45,162783.0,36583.0,9588.0,0.52,28.99,5.89,36.59,108.0,3.82,1.19,45.19,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,12.85,65.0,H-MC,3.42,194660.0,29990.0,37978.0,-0.09,18.21,19.51,41.28,89.0,0.79,1.42,27.65,X40N,ATH,AC
89,VOLTAS,1918.49,1.86,68.0,H-MC,5.39,213825.0,22083.0,73941.0,0.96,11.52,34.58,50.08,99.0,0.30,1.56,19.10,XY25,ATH,AC
32,HAVELLS,2069.16,0.20,63.0,H-MC,2.62,248383.0,652.0,74416.0,1.14,0.26,29.96,30.30,92.0,0.01,1.81,13.59,X40,ATH,ELECTRICAL
34,HEROMOTOCO,5949.07,-1.09,74.0,H-MC,18.53,183172.0,32259.0,25040.0,-0.20,21.38,13.67,37.97,93.0,1.29,1.34,52.09,AR,ATH,AUTO


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-1.09,74.0,H-MC,18.53,183172.0,32259.0,25040.0,-0.20,21.38,13.67,37.97,93.0,1.29,1.34,52.09,AR,ATH,AUTO
90,WHIRLPOOL,2270.00,-41.04,55.0,M-SC,5.18,99652.0,8154.0,70593.0,0.91,8.91,70.84,86.07,223.0,0.12,0.73,44.71,XR,NTT,DURABLES
40,INDIAMART,4850.92,-51.28,47.0,H-SC,12.54,136024.0,12688.0,121075.0,-1.30,10.29,89.01,108.45,119.0,0.10,0.99,34.94,AR,ATH,MISC
54,NATIONALUM,247.44,-48.63,57.0,H-MC,1.19,102290.0,1786.0,28856.0,0.63,1.78,28.21,30.49,79.0,0.06,0.75,34.76,MH,ATH,METALS
91,WIPRO,420.00,-11.52,52.0,M-LC,7.15,155372.0,4427.0,105031.0,0.10,2.93,67.60,72.51,53.0,0.04,1.14,9.13,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-5.35,41.0,H-SC,16.37,88159.0,-12804.0,107334.0,0.71,-12.68,121.75,93.63,148.0,-0.12,0.64,31.31,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,31.42,93562.0,-19987.0,60039.0,3.20,-17.60,64.17,35.27,268.0,-0.33,0.68,128.99,XR,NTT,HOTELS
52,MASFIN,398.61,-18.25,50.0,H-SC,13.17,92730.0,-5250.0,26855.0,-0.31,-5.36,28.96,22.05,152.0,-0.20,0.68,34.79,XR,ATH,FINANCE
35,HINDALCO,756.01,-16.60,61.0,H-LC,8.17,114760.0,10448.0,6197.0,-0.33,10.02,5.40,15.96,9.0,1.69,0.84,28.61,X5K,ATH,METALS
83,UJJIVANSFB,60.00,84.36,45.0,H-SC,14.10,116478.0,-26001.0,45520.0,-0.39,-18.25,39.08,13.70,163.0,-0.57,0.85,37.48,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,31.42,93562.0,-19987.0,60039.0,3.20,-17.60,64.17,35.27,268.0,-0.33,0.68,128.99,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-1.09,74.0,H-MC,18.53,183172.0,32259.0,25040.0,-0.20,21.38,13.67,37.97,93.0,1.29,1.34,52.09,AR,ATH,AUTO
27,DMART,5201.00,-7.44,69.0,H-LC,13.94,231064.0,46583.0,23776.0,-0.41,25.25,10.29,38.14,38.0,1.96,1.69,38.53,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,12.85,65.0,H-MC,3.42,194660.0,29990.0,37978.0,-0.09,18.21,19.51,41.28,89.0,0.79,1.42,27.65,X40N,ATH,AC
60,RELAXO,1176.00,-32.62,60.0,H-SC,7.16,62522.0,-53016.0,88006.0,-0.25,-45.89,140.76,30.28,136.0,-0.60,0.46,22.92,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.34
1,25,42.54
2,50,72.12


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.65
SC    36.43
MC    25.92
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.77
X40      15.17
X40N     14.01
XY25     11.24
AR        9.15
XR        8.82
OX40N     7.17
X5K       2.24
X200      1.71
MH        1.66
SR        1.06
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.61
H-MC    23.10
H-SC    21.86
M-SC    13.00
M-LC     7.00
M-MC     2.51
L-SC     1.57
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.91,-0.11,32.73
IT,13.11,-15.18,72.31
BANKS,7.15,-16.36,65.77
FINANCE,7.14,-23.37,68.86
PAINTS,5.86,-14.97,40.70
MISC,5.77,-16.27,82.92
ELECTRICAL,5.60,-7.35,66.52
AUTO,4.49,-9.97,57.81
AC,3.69,4.24,28.19


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2835847.0,23
AR,1227273.0,10
XR,1087668.0,13
X40,891532.0,10
X40N,516335.0,11
OX40N,477802.0,10
XY25,441126.0,8
SR,198302.0,2
X5K,131901.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2782252.0,27
M-SC,1466811.0,17
H-LC,1418331.0,18
H-MC,1216915.0,18
M-LC,420786.0,5
M-MC,334052.0,2
L-SC,246538.0,3
L-MC,59677.0,1
L-LC,42496.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,937350.0,6
M-SC,XY24,860007.0,7
H-SC,AR,666007.0,5
H-LC,X40,591274.0,5
H-SC,XR,527140.0,6
H-MC,XY24,478603.0,5
H-LC,AR,375638.0,2
M-MC,XY24,334052.0,2
H-MC,X40,219379.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
